<a href="https://colab.research.google.com/github/dajopr/lectures/blob/main/image_processing/lecture_04_thresholding_morphology_hough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise Sheet: Thresholding, Morphology, Connected Components & Hough Lines

**Goal:** Apply thresholding, morphological operations, connected components analysis, and Hough line detection to solve practical image analysis tasks.

## Setup

Run the following cells to import libraries, define helper functions, and set up image paths.
**Important:** Make sure you have the necessary image files (`document.png`, `shapes_noisy.png`, `barcode.png`, `coins.png`, `tennis.png`) in the specified `IMAGE_DIR`. If running in Colab, upload them to the `images` folder.

In [3]:
# Import necessary libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import math
# from google.colab.patches import cv2_imshow # Uncomment if using Colab
def show_images(
    images, titles, rows, cols, figsize=(15, 10), main_title="Image Comparison"
):
    """Helper function to display multiple images using Matplotlib"""
    if not images:
        print("No images to display.")
        return
    if len(images) != len(titles):
        print("Warning: Number of images and titles do not match.")
        min_count = min(len(images), len(titles))
        images = images[:min_count]
        titles = titles[:min_count]

    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    fig.suptitle(main_title, fontsize=16)

    if rows * cols == 1:
        axes = np.array([axes])
    axes = axes.ravel()

    plot_index = 0
    for i, img in enumerate(images):
        if plot_index < len(axes):
            current_ax = axes[plot_index]
            if img is not None:
                if len(img.shape) == 2 or (len(img.shape) == 3 and img.shape[2] == 1):
                    current_ax.imshow(img, cmap="gray")
                    current_ax.set_title(titles[i])
                elif len(img.shape) == 3 and img.shape[2] == 3:
                    current_ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                    current_ax.set_title(titles[i])
                elif len(img.shape) == 3 and img.shape[2] == 4:
                    current_ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA))
                    current_ax.set_title(titles[i])
                else:
                    current_ax.set_title(f"{titles[i]} (Unsupported Format)")
            else:
                current_ax.set_title(f"{titles[i]} (None)")
            current_ax.axis("off")
            plot_index += 1
        else:
            print(
                f"Warning: More images provided than subplot slots ({rows * cols}). Skipping '{titles[i]}'."
            )

    for j in range(plot_index, len(axes)):
        axes[j].axis("off")

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

print("Libraries imported.")

Libraries imported.


In [8]:
IMAGE_DIR = "./images"  # Assumes an 'images' folder in the same directory as the script/notebook

IMG_PATH_DOC = os.path.join(IMAGE_DIR, "document.png")
IMG_PATH_NOISY = os.path.join(IMAGE_DIR, "shapes_noisy.png")
IMG_PATH_BARCODE = os.path.join(IMAGE_DIR, "barcode.png")
IMG_PATH_COINS = os.path.join(IMAGE_DIR, "coins.png")
IMG_PATH_TENNIS = os.path.join(IMAGE_DIR, "tennis.jpg")

# --- Check if image files exist ---
required_image_paths = [
    IMG_PATH_DOC,
    IMG_PATH_NOISY,
    IMG_PATH_BARCODE,
    IMG_PATH_COINS,
    IMG_PATH_TENNIS,
]
all_files_found = True
if not os.path.exists(IMAGE_DIR):
    print(
        f"ERROR: Image directory '{IMAGE_DIR}' not found. Please create it and add images."
    )
    all_files_found = False
else:
    print(f"Checking for images in '{IMAGE_DIR}'...")
    for img_path in required_image_paths:
        if not os.path.exists(img_path):
            print(f"ERROR: Image file not found: {img_path}")
            all_files_found = False

if all_files_found:
    print("All required image files seem to be present.")
else:
    print(
        "\n!!! Please fix the missing image file issues before proceeding! Upload/place the required images. !!!"
    )
def load_image(path, mode=cv2.IMREAD_COLOR):
    if not all_files_found and not os.path.exists(path):
        print(f"Cannot load image {path}. File missing or directory issue.")
        return None
    img = cv2.imread(path, mode)
    if img is None:
        print(f"Error loading image {path} using OpenCV.")
    return img


print("Image setup complete.")

Checking for images in './images'...
All required image files seem to be present.
Image setup complete.


## Exercise 1: Comparing Thresholding Techniques

**Objective:** Apply and compare Global, Otsu's, and Adaptive thresholding methods on an image, potentially with uneven illumination.

**Instructions:**
1. Load the `document.png` image in grayscale.
2. Apply Global Thresholding using `cv2.threshold` with a manually chosen threshold value (e.g., 127). Use `cv2.THRESH_BINARY`.
3. Apply Otsu's Binarization using `cv2.threshold`. Use `cv2.THRESH_BINARY + cv2.THRESH_OTSU` (the threshold value argument will be ignored).
4. Apply Adaptive Thresholding using `cv2.adaptiveThreshold`. Try both `cv2.ADAPTIVE_THRESH_MEAN_C` and `cv2.ADAPTIVE_THRESH_GAUSSIAN_C`. Choose appropriate `blockSize` (e.g., 11, 15 - must be odd) and `C` (e.g., 2, 5).
5. Display the original grayscale image and all thresholded results using `show_images`.
6. Answer the discussion questions.

In [ ]:
print("\n--- Exercise 1: Comparing Thresholding ---")

img_doc = load_image(IMG_PATH_DOC, cv2.IMREAD_GRAYSCALE)
# --- Student Code Starts Here ---

# 1. Manual Global Thresholding
manual_thresh_val = 127
thresh_global = None # Replace with your code

# 2. Otsu's Binarization

thresh_otsu = None # Replace with your code

# 3. Adaptive Thresholding (Mean)
block_size_adapt = 15 # Must be odd
C_adapt = 2
thresh_adapt_mean = None # Replace with your result

# 4. Adaptive Thresholding (Gaussian)

thresh_adapt_gauss = None # Replace with your result

# --- Student Code Ends Here ---

# Display results
images_ex1 = [img_doc, thresh_global, thresh_otsu, thresh_adapt_mean, thresh_adapt_gauss]
titles_ex1 = ["Original Grayscale", f"Global (Thresh={manual_thresh_val})", "Otsu's", f"Adaptive Mean (Block={block_size_adapt}, C={C_adapt})", f"Adaptive Gaussian (Block={block_size_adapt}, C={C_adapt})"]
show_images(images_ex1, titles_ex1, 2, 3, main_title="Exercise 1: Thresholding Comparison") # Adjust rows/cols if needed


**Discussion Questions (Exercise 1):**
1. How does the result of Otsu's method compare to your manually chosen global threshold? When would Otsu's be particularly useful?
2. Compare the results of Adaptive Mean vs. Adaptive Gaussian thresholding. Are there noticeable differences?
3. Why is Adaptive Thresholding often better than Global Thresholding for images like documents with potentially uneven lighting? What do the `blockSize` and `C` parameters control?

## Exercise 2: Morphological Operations for Noise Removal

**Objective:** Use Opening and Closing morphological operations to clean up "salt and pepper" noise in a binary image.

**Instructions:**
1. Load the `noisy_shapes.png` image. Assume it's already binary or threshold it if necessary (Otsu's might work well if it's grayscale with noise).
2. Define a structuring element (kernel), for example, a 3x3 rectangle using `cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))`.
3. Apply Morphological Opening (`cv2.MORPH_OPEN`) to the noisy image using `cv2.morphologyEx`. This should primarily remove "salt" (white) noise.
4. Apply Morphological Closing (`cv2.MORPH_CLOSE`) to the noisy image using `cv2.morphologyEx`. This should primarily remove "pepper" (black) noise/holes.
5. Apply Opening *followed by* Closing to the noisy image. Does this handle both types of noise effectively?
6. Display the original noisy image and the results of Opening, Closing, and Opening+Closing using `show_images`.
7. Answer the discussion questions.

In [ ]:
img_noisy_gray = load_image(IMG_PATH_NOISY, cv2.IMREAD_GRAYSCALE)


# Assuming the loaded image might need thresholding to be truly binary
ret, img_noisy_bin = cv2.threshold(img_noisy_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
print("Applied Otsu's threshold to noisy image.")

# --- Student Code Starts Here ---
# 1. Define a structuring element (kernel)
kernel_size = 3
kernel = None # Replace with your kernel

if kernel is not None:
    # 2. Apply Morphological Opening
    opened_img = None # Replace with your result

    # 3. Apply Morphological Closing

    closed_img = None # Replace with your result

    # 4. Apply Opening then Closing

    opened_then_closed_img = None # Replace with your result
else:
    print("Kernel not defined, cannot perform morphological operations.")
    opened_img, closed_img, opened_then_closed_img = None, None, None


# --- Student Code Ends Here ---

# Display results
images_ex2 = [img_noisy_bin, opened_img, closed_img, opened_then_closed_img]
titles_ex2 = ["Original Noisy Binary", f"Opened (Kernel={kernel_size}x{kernel_size})", f"Closed (Kernel={kernel_size}x{kernel_size})", "Opened then Closed"]
show_images(images_ex2, titles_ex2, 2, 2, main_title="Exercise 2: Morphological Noise Removal")


**Discussion Questions (Exercise 2):**
1. Explain what Morphological Opening primarily removes and why (in terms of erosion followed by dilation).
2. Explain what Morphological Closing primarily removes/fills and why (in terms of dilation followed by erosion).
3. Did applying Opening followed by Closing work better than either operation alone for this type of noise? Why or why not? How does changing the `kernel_size` affect the results?

## Exercise 3: Counting Objects using Connected Components

**Objective:** Count the number of coins in an image using thresholding, optional morphological cleaning, and connected components analysis.

**Instructions:**
1. Load the `coins.png` image in grayscale.
2. Threshold the image to create a binary representation where coins are foreground (white) and the background is black. Otsu's method is often suitable here.
3. **Recommended:** Apply Morphological Opening (`cv2.MORPH_OPEN`) and/or Closing (`cv2.MORPH_CLOSE`) to the binary image to remove any small noise spots that might be counted as objects.
4. Apply `cv2.connectedComponentsWithStats` to the cleaned binary image.
5. Determine the number of coins by counting the number of labels found, remembering to **exclude the background label (label 0)**.
6. Iterate through the `stats` array (from label 1 onwards). For each coin component, get its bounding box `(x, y, w, h)` and draw the box on the original *color* image (load it separately or convert grayscale). You can also draw the component label number near the box using `cv2.putText`.
7. Display the thresholded image and the final image with bounding boxes and the count.
8. Answer the discussion questions.

In [ ]:
img_coins_gray = load_image(IMG_PATH_COINS, cv2.IMREAD_GRAYSCALE)
img_coins_color = load_image(IMG_PATH_COINS, cv2.IMREAD_COLOR) # For drawing

img_draw_ex3 = img_coins_color.copy()

# --- Student Code Starts Here ---

# 1. Threshold the image
# Note: If coins are darker than background, THRESH_BINARY_INV might be needed with Otsu.
# Or, if coins are bright, THRESH_BINARY with Otsu.

# If Otsu makes coins black and background white, invert it:
# if np.mean(thresh_coins) > 127: # Heuristic: if most of the image is white after Otsu
#    thresh_coins = cv2.bitwise_not(thresh_coins)
thresh_coins = None # Replace

# 2. Optional: Morphological Opening to remove noise

binary_for_cc = thresh_coins # Use thresh_coins if no morphology

num_coins = 0 # Initialize count

if binary_for_cc is not None:
    # 3. Connected Components Analysis

    num_labels, labels_map, stats, centroids = 0, None, None, None # Replace

    if stats is not None:
        # 4. Count components (excluding background label 0)
        num_coins = None # Replace

        # 5. Draw bounding boxes and labels, filter by area
        print(f"Found {num_labels-1} potential coin(s) before area filtering. Drawing boxes...")
        min_coin_area = 200 # Adjust based on image and coin size
        actual_coin_count = 0
        for i in range(1, num_labels): # Iterate from 1 to exclude background
            x = stats[i, cv2.CC_STAT_LEFT]
            y = stats[i, cv2.CC_STAT_TOP]
            w = stats[i, cv2.CC_STAT_WIDTH]
            h = stats[i, cv2.CC_STAT_HEIGHT]
            area = stats[i, cv2.CC_STAT_AREA]

            if area >= min_coin_area:
                actual_coin_count += 1
                cv2.rectangle(img_draw_ex3, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(img_draw_ex3, str(actual_coin_count), (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)
            else:
               print(f"  Component {i} rejected due to small area: {area}")


        num_coins = actual_coin_count # Update count after filtering
        print(f"Final count after area filtering: {num_coins}")
    else:
        print("Connected components analysis failed.")
else:
    print("Binary image for connected components is not available.")


# --- Student Code Ends Here ---

# Display results
images_ex3 = [img_coins_color, binary_for_cc if binary_for_cc is not None else np.zeros_like(img_coins_gray), img_draw_ex3]
titles_ex3 = ["Original Coins", "Binary Image for CC", f"Detected Coins: {num_coins}"]
show_images(images_ex3, titles_ex3, 1, 3, main_title="Exercise 4: Counting Coins")


**Discussion Questions (Exercise 3):**
1. Why might you need `THRESH_BINARY_INV` sometimes when using Otsu's method, or why might you need to invert the result of Otsu's? (Hint: Consider whether the objects or the background are brighter).
2. How did Morphological Opening (if used) help improve the accuracy of the count?
3. What problem would occur if two coins were touching in the image? How might you attempt to separate them using morphological operations learned previously (e.g., Erosion, Watershed algorithm - though Watershed is more advanced)?


## Exercise 4: Hough Line Transform for Line Detection

**Objective:** Detect straight lines in tennis court image using the Probabilistic Hough Line Transform.

**Instructions:**
1. Load the `road_lanes.png` image in grayscale.
2. Apply Canny edge detection (`cv2.Canny`) to find edges. You may need to tune the Canny thresholds (`threshold1`, `threshold2`).
3. Apply the Probabilistic Hough Line Transform (`cv2.HoughLinesP`) to the Canny edge map. Experiment with the parameters:
   * `rho`: Distance resolution (e.g., 1 pixel).
   * `theta`: Angle resolution (e.g., `np.pi/180` for 1 degree).
   * `threshold`: Minimum number of votes (intersections in Hough accumulator).
   * `minLineLength`: Minimum length of a line segment to be detected.
   * `maxLineGap`: Maximum allowed gap between points on the same line.
4. If lines are detected (`lines` is not None), iterate through the line segments and draw them onto the *original color image* (load it separately) using `cv2.line`. The lines array contains `[[x1, y1, x2, y2]]` for each segment.
5. Display the Canny edge map and the original image with detected lines drawn on it.
6. Answer the discussion questions.


In [ ]:
img_tennis_gray = load_image(IMG_PATH_TENNIS, cv2.IMREAD_GRAYSCALE)
img_tennis_color = load_image(IMG_PATH_TENNIS, cv2.IMREAD_COLOR) # For drawing


img_draw_ex5 = img_tennis_color.copy()

# --- Student Code Starts Here ---

# 1. Apply Canny Edge Detection (Tune thresholds)

canny_low_thresh = None
canny_high_thresh = None

edges = cv2.Canny(img_tennis_gray, canny_low_thresh, canny_high_thresh)

lines_p = None # Initialize lines
if edges is not None:
    # 2. Apply Probabilistic Hough Line Transform (Tune parameters)
    rho_acc = None         # Replace Rho accuracy: 1 pixel
    theta_acc = None       # Replace Theta accuracy: 1 degree
    hough_thresh = None    # Replace Min votes: Tune this
    min_line_len = None    # Replace Min length: Tune this
    max_line_gap = None    # Replace Max gap: Tune this

    lines_p = cv2.HoughLinesP(edges, rho_acc, theta_acc, hough_thresh,
                            minLineLength=min_line_len, maxLineGap=max_line_gap)

    # 3. Draw detected lines
    if lines_p is not None:
        print(f"Detected {len(lines_p)} line segments. Drawing...")
        for line in lines_p:
            x1, y1, x2, y2 = None # Extract coordinates # Replace
            cv2.line(img_draw_ex5, (x1, y1), (x2, y2), (0, 0, 255), 2) # Draw red lines
    else:
        print("No line segments detected with current parameters.")
else:
    print("Canny edge detection failed or produced no edges.")

# --- Student Code Ends Here ---

# Display results
images_ex4 = [img_tennis_color, edges if edges is not None else np.zeros_like(img_tennis_gray), img_draw_ex5]
titles_ex4 = ["Original Road", "Canny Edges", "Detected Lines (HoughP)"]
show_images(images_ex4, titles_ex4, 1, 3, main_title="Exercise 4: Hough Line Detection")


**Discussion Questions (Exercise 4):**
1. Explain the purpose of applying Canny edge detection *before* the Hough Transform.
2. How does changing the `threshold` parameter in `cv2.HoughLinesP` affect the number of lines detected?
3. What is the effect of increasing `minLineLength`? What about increasing `maxLineGap`?

## Optional Exercise: Refining Image Segmentation

Objective: Segment and filter regions of interest from an image using thresholding, morphological operations, and connected component analysis.

Image: Use the img_gray_ex3 image (ensure it's loaded as grayscale). If not available, load a grayscale image of your choice with some distinct structures.


Instructions:

1. Thresholding:
    - Apply cv2.adaptiveThreshold to img_gray_ex3 to create a binary image img_binary. Experiment with the blockSize (e.g., 35, 57, 75) and C (e.g., 5, 8, 10) parameters.
    - (Optional): Also try cv2.threshold with cv2.THRESH_OTSU.
        Use cv2.THRESH_BINARY_INV if your objects of interest are darker than the background in the original image and you want them white in the mask.
2. Morphological operations
    - Refine img_binary using cv2.morphologyEx. Start with cv2.MORPH_CLOSE.
    - Experiment with different kernel shapes and sizes for the structuring element (e.g., np.ones((H,W), np.uint8)). Try a vertical kernel (e.g., (7,1)), a horizontal kernel (e.g., (1,7)), and a square kernel (e.g., (3,3) or (5,5)).
3. Connected Components Analysis
    - Find connected regions in img_morph using cv2.connectedComponentsWithStats. This will give you num_labels, a labels matrix, stats, and centroids.
4. Region Filtering

    - Filter the detected regions based on their properties stored in stats. For example, filter by aspect ratio (height/width) or area (cv2.CC_STAT_AREA).
    - Create a new image labels_filtered that only contains the regions meeting your criteria (set others to 0, the background label). The original code filtered out regions where (height / width) < 5 was NOT true (i.e., it kept regions where height/width < 5). Modify this to, for example, keep regions that are significantly taller than they are wide, or regions within a specific area range.


In [ ]:
img_gray_opt = load_image("images/barcode.png", cv2.IMREAD_GRAYSCALE)

# Your code here

**Discussion Questions:**

1. Thresholding Impact: How did changing the blockSize and C parameters in cv2.adaptiveThreshold affect your img_binary? When might adaptive thresholding be more beneficial than a global method like Otsu's, and vice-versa?
2. Morphology Choices: Explain the effect of using a cv2.MORPH_CLOSE operation. How did different kernel shapes (e.g., vertical vs. square) and sizes influence the img_morph result for your specific image? When would you choose MORPH_OPEN instead?
3. Filtering Rationale: Describe the criteria you used for filtering connected components in Step 4. Why did you choose these criteria, and how might they need to change if you were looking for different types of objects in the image?